In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

1.3.0


In [7]:
from utils.reader import ptb_raw_data
from utils.batcher import ptb_batcher
from utils.conditional_scope import cond_name_scope, cond_variable_scope
from utils.unrolled_rnn import make_rnn_variables
from utils.unrolled_rnn import make_rnn_outputs
from utils.unrolled_rnn import make_summary_nodes
from utils.unrolled_rnn import make_placeholders

In [8]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [10]:
VOCAB_SIZE=128
EMBEDDING_SIZE=8
HIDDEN_SIZE=12
BATCH_SIZE=4
NUM_STEPS=16

tf.reset_default_graph()
placeholders = make_placeholders(
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS
)
rnn_vars = make_rnn_variables(
    vocab_size=VOCAB_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
)
rnn_outputs = make_rnn_outputs(
    input_sequence=placeholders['inputs'],
    vocab_size=VOCAB_SIZE,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS,
    rnn_variables=rnn_vars
)
summary_nodes = make_summary_nodes(
    targets=placeholders['targets'],
    logits=rnn_outputs['logits'],
)

In [12]:
inputs = np.random.randint(VOCAB_SIZE, size=(BATCH_SIZE, NUM_STEPS))
targets = np.random.randint(VOCAB_SIZE, size=(BATCH_SIZE, NUM_STEPS))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        outputs = sess.run(
            summary_nodes,
            feed_dict={
                placeholders['inputs']: inputs,
                placeholders['targets']: targets,
            }
        )
        if i % 100 == (100 - 1):
            print(outputs['loss'])
    params = sess.run(
        rnn_vars
    )
    print(params)

3.8984
3.3652
2.9219
2.543
2.0977
1.7021
1.6836
0.80908
0.43042
0.28394
{'embedding_matrix': array([[-0.0082, -0.0454, -0.0334, ..., -0.0219,  0.0055, -0.0169],
       [ 0.0243,  0.0396, -0.0424, ..., -0.0456, -0.0196, -0.0444],
       [ 0.035 ,  0.0081, -0.0045, ...,  0.0267,  0.0029,  0.0143],
       ..., 
       [-0.0247,  0.0205,  0.027 , ..., -0.0186, -0.0492,  0.0186],
       [-0.0403,  0.0278, -0.0484, ...,  0.0193,  0.0011,  0.0126],
       [ 0.0356,  0.0331, -0.0018, ...,  0.0067,  0.0422, -0.0382]], dtype=float16), 'gru_params': {'U_z': array([[  3.2422e-01,  -3.8306e-01,   3.1812e-01,  -8.7305e-01,   7.9163e-02,  -4.2017e-01,   1.3164e+00,   8.6475e-01,  -4.3359e-01,  -7.8223e-01,   1.7798e-01,   6.3330e-01],
       [  1.3879e-01,  -4.3286e-01,  -1.1953e+00,  -3.9624e-01,  -1.2578e+00,  -4.9683e-01,  -5.7227e-01,   4.0015e-01,  -6.8701e-01,   1.2617e+00,  -1.3701e+00,  -7.5146e-01],
       [  1.3623e-01,   4.2000e-03,  -9.2239e-03,  -4.3555e-01,  -6.1914e-01,  -2.7222e-02,  